In [67]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy model to device

This notebook explains the workflow for deploying a AI model to edge AI devices using "**Console for AITRIOS**". 

Instructions are described in [README.md](./README.md).

## Imports

In [68]:
import errno
import json
from pathlib import Path

import jsonschema
import numpy as np
import pandas as pd
from console_access_library.client import Client
from IPython.display import display

## Load Configurations
Load the configuration file and set the variables.

In [69]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

# Set values
should_create_deploy_config = json_load["should_create_deploy_config"]

if should_create_deploy_config is True:
    config_param = {}
    config_param["config_id"] = json_load["config_id"]
    if "create_config" in json_load:
        config_param["model_id"] = json_load["create_config"]["model_id"]
        if "model_version_number" in json_load["create_config"]:
            config_param["model_version_number"] = json_load["create_config"][
                "model_version_number"
            ]
        if "comment" in json_load["create_config"]:
            config_param["comment"] = json_load["create_config"]["comment"]

deploy_param = {}
deploy_param["config_id"] = json_load["config_id"]
device_ids_list = json_load["device_ids"]
device_ids = ",".join(device_ids_list)
deploy_param["device_ids"] = device_ids
if "replace_model_id" in json_load:
    deploy_param["replace_model_id"] = json_load["replace_model_id"]
if "comment" in json_load:
    deploy_param["comment"] = json_load["comment"]

## Load an instance of "**Console Access Library**" client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [70]:
%store -r
client_obj: Client = client_obj

## Create a deploy configuration

In [ ]:
# Get an instance of deploy API
deployment_obj = client_obj.get_deployment()

if should_create_deploy_config is True:
    # Call an API to create deploy configuration
    try:
        response = deployment_obj.create_deploy_configuration(**config_param)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print("Deploy configuration was created.")


## Deploy model to device
Start deploying to devices by running the following code :

In [ ]:
# Call an API to deploy the model to device
try:
    response = deployment_obj.deploy_by_configuration(**deploy_param)
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Start to deploy the model.")

## Check deployment status
Deploying to devices takes time.

To complete the deployment, ensure that the deployment status is **`Success`**.

After you start the deployment, run the following code cell to check the status :

In [ ]:
# Deploy status on Console
deploy_status_dictionary = {
    "0": "Deploying",
    "1": "Success",
    "2": "Fail",
    "3": "Cancel",
}
# Model status on Console
model_status_dictionary = {
    "0": "Waiting for execution",
    "1": "Deploying",
    "2": "Success",
    "3": "Fail",
}

deploy_ids = []
config_ids = []
deploy_statuses = []
model_statuses = []
update_dates = []
device_id_table = []
for device_id in device_ids_list:
    # Call an API to get deploy history
    try:
        response = deployment_obj.get_deploy_history(device_id)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # Create an output table
    deploys = response.get("deploys", [])
    cnt = 0
    for deploy in deploys:
        model = deploy.get("model", {})
        model_target_flg = model.get("model_target_flg", "")
        config_id = deploy.get("config_id", "")
        if model_target_flg == "1" and json_load["config_id"] == config_id:
            # Set device id
            if cnt == 0:
                device_id_table.append(device_id)
            else:
                # Fill a cell with a NAN
                device_id_table.append(np.NaN)
            # Set deploy ID
            deploy_id = deploy.get("id", "")
            deploy_ids.append(deploy_id)
            # Set config ID
            config_ids.append(config_id)
            # Set deploy status
            deploy_status = deploy.get("deploy_status", "")
            deploy_statuses.append(
                deploy_status_dictionary.get(
                    deploy_status, "Unknown status '" + deploy_status + "'"
                )
            )
            # Set model status
            model_status = model.get("model_status", "")
            model_statuses.append(
                model_status_dictionary.get(
                    model_status, "Unknown status '" + model_status + "'"
                )
            )
            # Set update date
            update_dates.append(deploy.get("upd_date", ""))

            cnt += 1
            # Display up to 5 deployment results
            if cnt == 5:
                break

if len(deploy_ids) == 0:
    raise Exception("There is no data in the deploy history list.")

output_frame = pd.DataFrame(
    {
        "device_id": device_id_table,
        "deploy_id": deploy_ids,
        "config_id": config_ids,
        "deploy_status": deploy_statuses,
        "model_status": model_statuses,
        "update_date": update_dates,
    }
)
output_frame = output_frame.fillna("-")
# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)